In [ ]:
import numpy as np
from mpl_toolkits import mplot3d
import open3d as o3d
import glob
import os
import os.path
import statistics as stats
from operator import itemgetter
from sklearn.cluster import SpectralClustering, KMeans, AffinityPropagation
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import copy

# Add all the linear method together, itterate and cluster, output lettuce

In [ ]:
# Load in point cloud
plant = 'plant3_bb_sim.ply'

# Output
plant_points = []

# Read .ply file
pcd_whole = o3d.io.read_point_cloud(plant)
ar_whole_plant = np.asarray(pcd_whole.points)

# Find middle z
z_axis = []

for i in ar_whole_plant:
    z_axis.append(i[2])

median_z = stats.median(z_axis)


# Slicing

slice_storage_list = []

a = ar_whole_plant[0][0]
cnt = 1

# Create one slice
for i in ar_whole_plant:
    
    
    if (abs(i[0] - a) < 10) | (len(slice_storage_list) < 1):
        slice_storage_list.append(list(i))
        a = i[0]

        # print(f'Building intermed list: {len(intermed_list)}')






    # Cluster the slice, and add clusters that have  a point above the overall
    # z to a 'plant list'
    else:
        # Drop y to flatten here
        flat_slice = copy.deepcopy(slice_storage_list)
        for j in flat_slice:
            del j[1]
        # Cluster here
        km = KMeans(
            n_clusters=5, init='random',
            n_init=10, max_iter=300, 
            tol=1e-04, random_state=0
        )
        
        X = np.array(flat_slice)
        y_km = km.fit_predict(X)

        # Choose cluster
        l0 = []
        l1 = []
        l2 = []
        l3 = []
        l4 = []
        for index, row in enumerate(slice_storage_list):
            # print(index, y_km[index], row)


            if y_km[index] == 0:
                l0.append(row)
            if y_km[index] == 1:
                l1.append(row)
            if y_km[index] == 2:
                l2.append(row)
            if y_km[index] == 3:
                l3.append(row)
            if y_km[index] == 4:
                l4.append(row)

        clusters = [l0,l1,l2,l3,l4]

        

        for c in clusters:
            for i in c:
                # print(i)
                # Below worked ok
                # if i[2] > median_z + (median_z * .01):
                if i[2] > median_z + (median_z * .015):
                    plant_points.extend(c)

                    
                    break

        # To next slice
        # cnt+=1
        # pcd_slice = o3d.geometry.PointCloud()
        # pcd_slice.points = o3d.utility.Vector3dVector(intermed_list)
        # linear_matrix.append(intermed_list)

        slice_storage_list = []

        a = i[0]
        print(len(plant_points))
        # print(f'Slice {cnt} done')
        cnt+=1

# Output plant pcd
pcd_slice = o3d.geometry.PointCloud()
pcd_slice.points = o3d.utility.Vector3dVector(plant_points)
o3d.io.write_point_cloud(f'/home/travis_s/data/test_plys/plant_slice.ply', pcd_slice)

In [ ]:
pcd_slice